In [1]:
import mysql.connector
import getpass  # use dotenv, import dotenv
import pandas as pd
import tqdm
import json
import requests

from sql_utils import insert_to_table

In [2]:
# connect to my local mysql server
my_pokemon = mysql.connector.connect(
    host="localhost", 
    user="root", 
    password=getpass.getpass("password"),
    port=3306, 
    database="RAW_POKEMON"
)

my_cursor = my_pokemon.cursor(buffered=True)

In [3]:
# insert types
def insert_types(connector, cursor, jsonpath): 
    with open(jsonpath, 'r') as f:
        values = json.load(f)
    for value in values: 
        insert_to_table(
            connector=connector, 
            cursor=cursor,
            table='Types_map',
            columns=['type_names'],  # these 2 have to be in list of items
            values=[value]
        )


insert_types(connector=my_pokemon, cursor=my_cursor, jsonpath='./raw/types.json')

In [4]:
# completing the table, ETL (treating the xlsx as the extracted data, then transform into the format to match the table)
excel_path = "./raw/data.xlsx"
all_pokemons = pd.read_excel(excel_path, index_col=[0])


# transform
rows = []
keep_columns = ['Id', 'Name', 'Type(s)', 'HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed']
for i, row in all_pokemons.iterrows():
    row = [
        v if k != 'Type(s)' else eval(v) 
        for k, v in row.items() 
        if k in keep_columns
    ]  # use list here as tuple is immutable
    # deal with the type
    if len(row[2]) == 1:  # only 1 type
        types = row.pop(2)
        row.insert(2, types[0])
        row.insert(3, "Null")
    elif len(row[2]) == 2:  # 2 types
        types = row.pop(2)
        row.insert(2, types[0])
        row.insert(3, types[1])
    rows.append(row)


keys = ['pokedex_id', 'pokemon_name', 'type1', 'type2', 'HP',
        'Attack', 'Defense', 'Special_Attack', 'Special_Defense', 'Speed']
for row in tqdm.tqdm(rows):
    insert_to_table(my_pokemon, my_cursor, table='Pokemons',
                    columns=keys, values=row)

100%|██████████| 1045/1045 [00:00<00:00, 2372.34it/s]


In [5]:
# insert alias
with open('./raw/alias.json', 'r') as f:
    alias = json.load(f)

for k, v in alias.items(): 
    insert_to_table(
        connector=my_pokemon, 
        cursor=my_cursor,
        table='Alias_map',
        columns=['name1', 'name2'],  # these 2 have to be in list of items
        values=[v, k]
    )

In [6]:
# insert battles
battle_path = './raw/match_data.json'
battle = pd.read_json(battle_path)

# have a look
keys = ['pokemon1', 'pokemon2', 'outcome', 'log_url']
for i, row in tqdm.tqdm(battle.iterrows()):
    insert_to_table(my_pokemon, my_cursor, table='Battles',
                    columns=keys, values=[v for k, v in row.items()])

8895it [00:03, 2528.28it/s]
